<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IEEE-CIS-Fraud-Detection" data-toc-modified-id="IEEE-CIS-Fraud-Detection-1">IEEE-CIS Fraud Detection</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1">Imports</a></span></li><li><span><a href="#Reading-in-and-merging-dataframes" data-toc-modified-id="Reading-in-and-merging-dataframes-1.2">Reading in and merging dataframes</a></span></li><li><span><a href="#Data-exploration-and-preprocessing" data-toc-modified-id="Data-exploration-and-preprocessing-1.3">Data exploration and preprocessing</a></span><ul class="toc-item"><li><span><a href="#Random-sampling-of-the-test-set" data-toc-modified-id="Random-sampling-of-the-test-set-1.3.1">Random sampling of the test set</a></span></li><li><span><a href="#Simplfication-of-data-set" data-toc-modified-id="Simplfication-of-data-set-1.3.2">Simplfication of data set</a></span></li></ul></li><li><span><a href="#Data-processing-steps" data-toc-modified-id="Data-processing-steps-1.4">Data processing steps</a></span><ul class="toc-item"><li><span><a href="#Assign-order-to-ordinals" data-toc-modified-id="Assign-order-to-ordinals-1.4.1">Assign order to ordinals</a></span></li></ul></li><li><span><a href="#Handling-dates" data-toc-modified-id="Handling-dates-1.5">Handling dates</a></span></li></ul></li></ul></div>

# IEEE-CIS Fraud Detection

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
import multiprocessing as mp
import torch
from functools import partial
pd.options.display.max_columns = None

In [3]:
path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')

In [4]:
Path.BASE_PATH = path

In [5]:
# with ZipFile('ieee-fraud-detection.zip', 'r') as zip_ref:
#     zip_ref.extractall(path/'Data')

## Reading in and merging dataframes

In [6]:
files = [path/'test_identity.csv', 
         path/'test_transaction.csv',
         path/'train_identity.csv',
         path/'train_transaction.csv']

In [7]:
%%time
def read_data(file):
    return pd.read_csv(file, low_memory=False)

with mp.Pool() as pool:
    test_id, test_tr, train_id, train_tr = pool.map(read_data, files)   

CPU times: user 6.12 s, sys: 14.8 s, total: 20.9 s
Wall time: 2min 23s


In [8]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
test = pd.merge(test_tr, test_id, on='TransactionID', how='left')

In [9]:
train.to_csv(path/'train.csv')
test.to_csv(path/'test.csv')

Now I will create take a sample of this dataset to work with during experimentation.

## Data exploration and preprocessing

### Random sampling of the test set

In [10]:
# randomly sample the date without replacement
train_s = train.sample(frac=0.1, axis=0)   

In [11]:
assert train_s.columns.shape[0]==train.columns.shape[0] 

In [12]:
train_s.to_csv(path/'train_s.csv')

### Simplfication of data set

In [13]:
train_s = pd.read_csv(path/'train_s.csv', index_col=[0], low_memory=False)

In [14]:
train_s.shape

(59054, 434)

In [15]:
train_s.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

To simplify the dataset I will drop every collumn that records a Vxxx feature. There are over 339 such features many of which are highkly correllated.

In [16]:
train_ss = train_s.drop(list(train_s.filter(regex = 'V')), axis = 1)

In [17]:
assert len(train_ss.columns)==len(train_s.columns) - 339

In [18]:
train_ss.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9', 'id_01', 'id_02', 'id_03', 'id_04',
       'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12',
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20',
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28',
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38', 'DeviceType', 'DeviceInfo'],
      dtype='object')

In [19]:
train_ss.to_csv(path/'train_ss.csv')

## Data processing steps

### Assign order to ordinals

There does not appear to be any identifiable ordinal variables amoung the catergorical variables in the dataset.

## Handling dates

The only place where there is datetime info is the TransactionDT collumn. It's possible to turn this delta into a datetime (assuming units are seconds as indicated by some on the disscussion board) by choosing a starting point. [example](https://www.kaggle.com/nroman/eda-for-cis-fraud-detection). I will train my first models without doing that.

Next need to add datepart (i think it's important for parts of this that the correct start date is used)